# Deep Research

In [1]:
from agents import Agent, WebSearchTool, trace, Runner, gen_trace_id, function_tool
from agents.model_settings import ModelSettings
from pydantic import BaseModel, Field
from dotenv import load_dotenv
import asyncio
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
from typing import Dict
from IPython.display import display, Markdown

In [2]:
# Load the environment variables
load_dotenv(override=True)

True

## OpenAI Hosted Tools

OpenAI Agents SDK includes the following hosted tools:

The `WebSearchTool` lets an agent search the web.  
The `FileSearchTool` allows retrieving information from your OpenAI Vector Stores.  
The `ComputerTool` allows automating computer use tasks like taking screenshots and clicking.

### Important note - API charge of WebSearchTool

This is costing me 2.5 cents per call for OpenAI WebSearchTool. That can add up to $2-$3 for the next 2 labs. We'll use free and low cost Search tools with other platforms, so feel free to skip running this if the cost is a concern. Also student Christian W. pointed out that OpenAI can sometimes charge for multiple searches for a single call, so it could sometimes cost more than 2.5 cents per call.

Costs are here: https://platform.openai.com/docs/pricing#web-search

In [4]:
# Define the instructions for the research agent
INSTRUCTIONS = "You are a research assistant. Given a search term, you search the web for that term and produce a concise summary of the results. The summary must 2-3 paragraphs and less than 300 words. Capture the main points. Write succintly, no need to have complete sentences or good grammar. This will be consumed by someone synthesizing a report, so it's vital you capture the essence and ignore any fluff. Do not include any additional commentary other than the summary itself."

# Create the search_agent Agent. Should use tools and model_settings
search_agent = Agent(
    name="Search agent",
    instructions=INSTRUCTIONS,
    tools=[WebSearchTool(search_context_size="low")],
    model="gpt-4o-mini",
    # This is how you tell the agent that it is required to run the tool. It is mandatory.
    model_settings=ModelSettings(tool_choice="required")
)

In [6]:
# Define a messsage for the search_agent
message = "Latest AI Agent frameworks in 2025"

with trace("Search"):
    result = await Runner.run(search_agent, message)

display(Markdown(result.final_output))

In 2025, several AI agent frameworks have emerged, each offering unique capabilities:

- **Agent Lightning**: A flexible framework enabling reinforcement learning-based training of large language models (LLMs) for any AI agent. It decouples agent execution from training, allowing seamless integration with existing agents developed through various methods. ([arxiv.org](https://arxiv.org/abs/2508.03680?utm_source=openai))

- **Polymorphic Combinatorial Framework (PCF)**: Utilizes LLMs and mathematical frameworks to design adaptive AI agents for complex, dynamic environments. PCF enables real-time parameter reconfiguration through combinatorial spaces, supporting scalable and ethical AI applications. ([arxiv.org](https://arxiv.org/abs/2508.01581?utm_source=openai))

- **Cognitive Kernel-Pro**: An open-source, multi-module agent framework democratizing the development and evaluation of advanced AI agents. It focuses on curating high-quality training data and enhancing agent robustness and performance. ([arxiv.org](https://arxiv.org/abs/2508.00414?utm_source=openai))

- **AutoAgent**: A fully automated, zero-code framework for LLM agents, enabling users to create and deploy agents through natural language alone. It serves as a versatile multi-agent system for general AI assistants. ([arxiv.org](https://arxiv.org/abs/2502.05957?utm_source=openai))

- **CrewAI**: Introduces collaborative multi-agent teams with role-based agent definitions, built-in memory, and communication between agents. It integrates with LangChain and LLMs, facilitating complex, multi-step processes across agents. ([linkedin.com](https://www.linkedin.com/pulse/top-10-frameworks-building-ai-agents-2025-rachel-grace-okrqc?utm_source=openai))

- **LlamaIndex**: Connects LLMs with custom data sources through intelligent indexing and retrieval systems, supporting document loaders for various formats and indexing strategies. ([linkedin.com](https://www.linkedin.com/pulse/top-10-frameworks-building-ai-agents-2025-rachel-grace-okrqc?utm_source=openai))

- **Microsoft Semantic Kernel**: A lightweight framework bridging traditional development with AI capabilities, emphasizing seamless integration into enterprise applications. It supports multiple languages and orchestrates multi-step AI workflows. ([linkedin.com](https://www.linkedin.com/pulse/top-5-frameworks-building-ai-agents-2025-sahil-malhotra-wmisc?utm_source=openai))

- **Amazon Bedrock AgentCore**: A platform designed to simplify the development and deployment of advanced AI agents, offering modular services supporting the full production lifecycle, including scalable serverless deployment and context management. ([techradar.com](https://www.techradar.com/pro/aws-looks-to-super-charge-ai-agents-with-amazon-bedrock-agentcore?utm_source=openai))

- **Model Context Protocol (MCP)**: An open standard introduced by Anthropic to standardize AI systems' integration and data sharing with external tools and data sources, adopted by major AI providers like OpenAI and Google DeepMind. ([en.wikipedia.org](https://en.wikipedia.org/wiki/Model_Context_Protocol?utm_source=openai))

These frameworks reflect the rapid evolution in AI agent development, emphasizing flexibility, collaboration, and seamless integration across diverse applications. 

#### We will now use Structured Outputs, and include a description of the fields

In [9]:
HOW_MANY_SEARCHES = 3

# Instructions for the planner_agent
INSTRUCTIONS = f"You are a helpful research assistant. Given a query, come up with a set of web searches to perform to best answer the query. Output {HOW_MANY_SEARCHES} terms to query for."

# Define a strcutured output class for a websearch item
# By annotating the fields, we ensure that the information is provided. This is provided to the model so it knows why it's populating these fields
# Chain of thought prompting
# The reason is placed before the query so that the next token predicted by the model is more consistent with the reason provided
class WebSearchItem(BaseModel):
    reason: str
    "Your reasoning for why this search is important to the query."

    query: str
    "The search term to use for the web search."

class WebSearchPlan(BaseModel):
    searches: list[WebSearchItem]
    """A list of web searches to perform to best answer the query."""

# Create the planner_agent
planner_agent = Agent(
    name="PlannerAgent",
    instructions=INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=WebSearchPlan
)

In [12]:
# Prompt for the planner_agent
message = "Latest AI Agent frameworks in 2025"

with trace("Search"):
    result = await Runner.run(planner_agent, message)
    print(result.final_output)

searches=[WebSearchItem(reason='To find recent developments and frameworks in AI agents for 2025.', query='latest AI agent frameworks 2025'), WebSearchItem(reason='To discover industry reports or articles that outline the advancements in AI agent technology for 2025.', query='AI agent technology advancements 2025'), WebSearchItem(reason='To identify popular or emerging AI agents and their frameworks used in various applications in 2025.', query='emerging AI agent frameworks applications 2025')]


In [28]:
@function_tool
def send_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("twostrokes210business@gmail.com") # Change this to your verified email
    to_email = To("twostrokes210business@gmail.com") # Change this to your email
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    response = sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

In [29]:
model_type = "gpt-4o-mini"

# Prompt for the email agent
EMAIL_PROMPT = """You are able to send a nicely formatted HTML email based on a detailed report.
You will be provided with a detailed report. You should use your tool to send one email, providing the 
report converted into clean, well presented HTML with an appropriate subject line."""

# Create the email_agent with the send_email as a tool
email_agent = Agent(
    name="Email Agent",
    instructions=EMAIL_PROMPT,
    tools=[send_email],
    model=model_type
)

In [30]:
REPORT_INSTRUCTIONS = (
    "You are a senior researcher tasked with writing a cohesive report for a research query. "
    "You will be provided with the original query, and some initial research done by a research assistant.\n"
    "You should first come up with an outline for the report that describes the structure and "
    "flow of the report. Then, generate the report and return that as your final output.\n"
    "The final output should be in markdown format, and it should be lengthy and detailed. Aim "
    "for 5-10 pages of content, at least 1000 words."
)

# Create a structured output for Report Data. Fields: short_summary, markdown_report, follow_up_questions. Provide description under
class ReportData(BaseModel):
    short_summary: str
    """A short 2-3 sentence summary of the findings."""

    markdown_report: str
    """The final report"""

    follow_up_questions: list[str]
    """Suggested topics to research further"""

# Create the writer_agent that outputs a ReportData object
writer_agent = Agent(
    name="WriterAgent",
    instructions=REPORT_INSTRUCTIONS,
    model=model_type,
    output_type=ReportData
)


#### The next 3 functions will plan and execute the search, using planner_agent and search_agent

In [31]:
# This function calls the planner_agent (In charge of coming up with a list of things to search)
async def plan_searches(query: str):
    """ Use the planner_agent to plan which searches to run for the query """
    print("Planning searches...")
    # Run the agent
    plan_search_result = await Runner.run(planner_agent, f"Query: {query}")
    print(f"Will perform {len(plan_search_result.final_output.searches)} searches")
    return plan_search_result.final_output

# This function will use the search agent to run a web search
async def search(item: WebSearchItem):
    """ Use the search agent to run a web search for each item in the search plan """
    input = f"Search term: {item.query}\nReason for searching: {item.reason}"
    # Run the agent
    result = await Runner.run(search_agent, input)
    return result.final_output

# This function will perform the actual searches that are returned in the WebSearchPlan list
async def perform_searches(search_plan: WebSearchPlan):
    """ Call search() for each item in the search plan """
    print("Searching...")
    num_completed = 0
    # Run tasks in parallel
    tasks = [asyncio.create_task(search(item)) for item in search_plan.searches]
    results = await asyncio.gather(*tasks)
    print("Finished searching")
    return results

#### The next 2 functions write a report and email it

In [32]:
async def write_report(query: str, search_results: list[str]):
    """ Use the writer agent to write a report based on the search results """
    print("Thinking about report")
    input = f"Original query: {query}\nSummarized search results: {search_results}"
    # Run the agent
    result = await Runner.run(writer_agent, input)
    print("Finished writing report")
    return result.final_output

async def send_email(report: ReportData):
    """ Use the email agent to send an email with the report """
    print("Writing email...")
    # Run the agent
    result = await Runner.run(email_agent, report.markdown_report)
    print("Email sent")
    return result

## Showtime!

In [33]:
query = "Latest AI Agent framewors in 2025"

with trace("Research trace"):
    print("Starting research...")
    # First call the search plan in order to create a lsit of search items
    search_plan = await plan_searches(query)

    # Call the perform_searches to perform the searches
    search_results = await perform_searches(search_plan)

    # Call to create the report based on the search results
    report = await write_report(query, search_results)

    # Send the email
    await send_email(report)


Starting research...
Planning searches...
Will perform 3 searches
Searching...
Finished searching
Thinking about report
Finished writing report
Writing email...
Email sent
